In [ ]:
import pandas as pd
from lxml import etree
import os
import glob
import numpy as np
import sys
import cv2
import random
import tqdm

In [ ]:
#Box class for defining object bounding box
class Box(object):
    def __init__(self):
        self.top = 0
        self.left = 0
        self.width = 0
        self.height = 0
    def display(self):
        print(self.top,self.left,self.width,self.height)

        
# Image class which has image address and its facial landmarks
class Image(object):
    def __init__(self):
        self.path = ""
        self.box = Box()
        self.landmarks = [[0 for x in range(2)] for y in range(8)] 
    def display(self):
        print(self.path)
        self.box.display()
        print(self.landmarks[0],self.landmarks[3])

In [ ]:
#Function to get bounding box of the face landmark points from image
def bounding_box_from_annotation():
    face_box = Box()
    face_box.left = 1
    face_box.top = 1
    face_box.width = 639
    face_box.height = 639    
    return face_box    

In [ ]:
def create_image_element(image):
    path = image.path
    image_e = etree.Element('image')
    image_e.set('file', str(path))
    faceBox = image.box
    box = etree.Element('box')

    box.set('top', str(int(faceBox.top)))
    box.set('left', str(int(faceBox.left)))
    box.set('width', str(int(faceBox.width)))
    box.set('height', str(int(faceBox.height)))
    
    image_e.append(box)
    #create landmarks for the bounding box
    for k in range(len(image.landmarks)):
        landmark = etree.Element('part')
        landmark.set('name',str(int(k)))
        landmark.set('x',str(int(image.landmarks[k][0])))
        landmark.set('y',str(int(image.landmarks[k][1])))
        box.append(landmark)   
    return image_e

In [ ]:
def create_dlib_xml(images):
    root = etree.Element('dataset')
    root.append(etree.Element('name'))
    root.append(etree.Element('comment'))

    images_e = etree.Element('images')
    root.append(images_e)
    
    for i in images:
        #i.display()
        images_e.append(create_image_element(i))

    et = etree.ElementTree(root)
    return et

In [ ]:
SaveDir = '/Users/paraskapoor/Desktop/CONCORDIA/Hackathons/DLIB_DATA/New/saved/'
XML_FILE = '/Users/paraskapoor/Desktop/CONCORDIA/Hackathons/DLIB_DATA/New/'
ImageDir = '/Users/paraskapoor/Desktop/CONCORDIA/Hackathons/DLIB_DATA/sample/'
AnnotationFile = '/Users/paraskapoor/Desktop/CONCORDIA/Hackathons/DLIB_DATA/sample/via_export_csv.csv'

In [ ]:
STANDARD_IMAGE_SHAPE = (640,640)

In [ ]:
import re
import csv

pattern = r"{\"name\":\"point\",\"cx\":(\d+),\"cy\":(\d+)}"
count = 0
DICT = {}
with open(AnnotationFile) as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        if count == 0:
            count = 1
            continue
        if len(row[5]) > 2:
            Points = row[5]
            m = re.search(pattern,Points)
            point = [float(m.groups()[0]),float(m.groups()[1])]
            filename = row[0]
            if filename in DICT.keys():
                DICT[filename].append(point)
            else:
                DICT[filename] = []
                DICT[filename].append(point)
print(DICT)

In [ ]:
IMAGES = []
image_index_global = 0
for filename in DICT.keys():
    framepath = ImageDir + filename
    
    image = cv2.imread(framepath)
    height,width,_ = image.shape
    
    image = cv2.resize(image,(640,640))
    
    framepath = SaveDir + filename
    cv2.imwrite(framepath,image)
    
    print(framepath)
    IMAGES.extend([Image()])
    IMAGES[image_index_global].path = framepath

    for idx in range(8):
        IMAGES[image_index_global].landmarks[idx][0] = int(DICT[filename][idx][0]*640/width)
        IMAGES[image_index_global].landmarks[idx][1] = int(DICT[filename][idx][1]*640/height)
    IMAGES[image_index_global].box = bounding_box_from_annotation()
    image_index_global = image_index_global + 1

In [ ]:
et = create_dlib_xml(IMAGES)
loc = XML_FILE + 'Training.xml'
et.write(file=loc,pretty_print=True)